# Compute RHI Scan Parameters

Tryout to see if we can compute the scan parameters for setting up a set of RHI scans

```
Input:
    radar (lon, lat)
    target (lon, lat)
    height
    heading

Output:
    elevation (minimum)
    azimuth
    range
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def deg2rad(x):
    return x * np.pi / 180.

def rad2deg(x):
    return x * 180. / np.pi

def coord2point(coord, h=0.):
    (lon, lat) = coord
    r = 6371. + h
    lon_rad = deg2rad(lon)
    lat_rad = deg2rad(lat)
    return np.array([
        r * np.cos(lat_rad) * np.sin(lon_rad),
        r * np.sin(lat_rad),
        r * np.cos(lat_rad) * np.cos(lon_rad),
    ], dtype=float)

def rotx(x):
    sx = np.sin(x)
    cx = np.cos(x)
    return np.array([[1., 0., 0.], [0., cx, -sx], [0., sx, cx]])


def roty(x):
    sx = np.sin(x)
    cx = np.cos(x)
    return np.array([[cx, 0., sx], [0., 1., 0.], [-sx, 0., cx]])

def rotz(x):
    sx = np.sin(x)
    cx = np.cos(x)
    return np.array([[cx, -sx, 0.], [sx, cx, 0.], [0., 0., 1.]])

def norm(x):
    return np.sqrt(np.sum(x ** 2))

def showvec3(p):
    (x, y, z) = p
    return f'{x:.2f} {y:.2f} {z:.2f}'

def params(target, radar, height=2.5, debug=False):
    pt = coord2point(target, h=height)

    ry = roty(deg2rad(-radar[0]))
    rx = rotx(deg2rad(radar[1]))
    m = np.matmul(rx, ry)

    yt = np.matmul(m, pt)

    d = yt[:2]
    r = norm(d)
    az = rad2deg(np.arctan2(d[0], d[1]))
    el = rad2deg(np.arctan2(height, r))

    if debug:
        pr = coord2point(radar)
        yr = np.matmul(m, pr)

        # zenith = pr / norm(pr)
        # east = np.cross([0, 1, 0], zenith)
        # east = east / norm(east)
        # north = np.cross(zenith, east)
        # north = north / norm(north)

        print('target @ ' + showvec3(yt))
        print('radar @ ' + showvec3(yr))
        print(f'el = {el:.2f}   az = {az:.2f}   r = {r:.2f}')

    return {
        'el': el,
        'az': az,
        'r': r
    }

In [ ]:
radar = (-90, 0)
target = (-90.1, 0.1)
# target = (-91, 36)
# target = (-90, -1)

# p = params(target, radar, debug=True)
p = params(target, radar)

print(p)

# Coordinate to 3D Points of the Earth

A convention similar to the RadarHub is used here where the screen is used as the display canvas where
 - x-axis to the right;
 - y-axis to the top; and
 - z-axis to the viewer

In [ ]:
points = []
for lat in range(-80, 80, 10):
    for lon in range(-180, 180, 10):
        x = coord2point((lon, lat))
        points.append(x)
pp = np.array(points)

In [ ]:
fig = plt.figure(figsize=(6.9, 6.9), dpi=96)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pp[:, 0], pp[:, 1], pp[:, 2], marker='.')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.grid(False)
ax.set_box_aspect([1, 1, 1])

In [ ]:
plt.close()